In [ ]:
!pip install fastai==0.7.0 --no-deps
!pip install torch==0.4.1 torchvision==0.2.1

In [ ]:
from fastai.conv_learner import *

In [ ]:
from fastai.plots import *

In [ ]:
from sklearn.metrics import f1_score

def f1(preds, targs, start=0.17, end=0.24, step=0.01):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return max([f1_score(targs, (preds>th), average='macro')
                    for th in np.arange(start,end,step)])

In [ ]:
metrics=[f1]
f_model = resnet34

In [ ]:
label_csv = '../input/train.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [ ]:
import pathlib
def get_data(sz, colour):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    data = ImageClassifierData.from_csv('../input', 'train', label_csv, tfms=tfms,
                    suffix='_{}.png'.format(colour), val_idxs=val_idxs, test_name='test')
    data.path = pathlib.Path('.')
    return data

In [ ]:
data = get_data(256, 'green')

In [ ]:
learn = ConvLearner.pretrained(f_model, data, metrics=metrics)
learn.opt_fn = optim.Adam

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.006
learn.fit(lr, 2, cycle_len=1, cycle_mult=2, wds=1e-3)

In [ ]:
lr = 0.004
lrs = np.array([lr/9,lr/3,lr])

learn.unfreeze()
learn.fit(lrs, 4, cycle_len=1, cycle_mult=2)#, wds=1e-4

In [ ]:
preds_t,y_t = learn.TTA(n_aug=8,is_test=True)
preds_t = np.stack(preds_t, axis=-1)
pred_t = preds_t.mean(axis=-1)

In [ ]:
def save_pred(pred, th=0.0, fname='protein_classification.csv'):
    pred_list = []
    for line in pred:
        s = ' '.join(list([str(i) for i in np.nonzero(line>th)[0]]))
        pred_list.append(s)
    
    df = pd.DataFrame({'Id':learn.data.test_ds.fnames,'Predicted':pred_list})
    df.sort_values(by='Id').to_csv(fname, header=True, index=False)

In [ ]:
save_pred(pred_t,0.01,'protein_classification_0.csv')

In [ ]:
FileLink('protein_classification_0.csv')

learn.set_data(get_data(64, 'green'))

lrf=learn.lr_find()
learn.sched.plot()

lr = 0.005
lrs = np.array([lr/9,lr/3,lr])

learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

learn.set_data(get_data(128, 'green'))

lrf=learn.lr_find()
learn.sched.plot()

lr = 0.005
lrs = np.array([lr/9,lr/3,lr])

learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

tfms.bs = 16

learn.sched.plot_loss()

learn.fit(lrs, 1, cycle_len=1, cycle_mult=2)

learn.set_data(get_data(256, 'green'))
learn.freeze()

learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

learn.sched.plot_loss()

f1(preds,y)

log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)

probs.shape

df = pd.DataFrame(probs)
df.columns = data.classes

df.insert(0, 'Id', [o[5:-4] for o in data.test_ds.fnames])

df.head()